In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [ ]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
openai_api_key = os.getenv('OPENAI_API_KEY')

print(openai_api_key)  # 출력된 API 키 확인

In [13]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key= openai_api_key

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'


Agent(에이전트) 생성하기

Agent를 정의하고, role(역할), goal(목표), backstory(배경 설명)를 제공합니다.

LLM(대규모 언어 모델)은 롤플레잉을 할 때 더 나은 성능을 보이는 것으로 확인되었습니다.

### Agent: 기획자(Planner)

**참고**: _여러 문자열_ 사용의 이점:
```Python
varname = "텍스트의 첫 번째 줄"
          "텍스트의 두 번째 줄"
```

_삼중 따옴표 독스트링_ 사용과 비교했을 때:
```Python
varname = """텍스트의 첫 번째 줄
             텍스트의 두 번째 줄
          """
```
여러 문자열을 사용하면 공백과 줄바꿈 문자가 추가되는 것을 피할 수 있어, LLM에 전달할 때 더 나은 형식을 유지할 수 있습니다.

In [14]:
planner = Agent(
    role="콘텐츠 기획자",
    goal="{topic}에 대한 흥미롭고 사실에 기반한 콘텐츠 기획",
    backstory="당신은 {topic}에 대한 블로그 글을 "
              "기획하는 일을 하고 있습니다. "
              "독자들이 새로운 것을 배우고 "
              "정보에 기반한 결정을 내릴 수 있도록 "
              "도움이 되는 정보를 수집합니다. "
              "당신의 작업은 콘텐츠 작성자가 "
              "이 주제로 글을 쓸 수 있는 토대가 됩니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [15]:
writer = Agent(
    role="콘텐츠 작성자",
    goal="{topic}에 대한 통찰력 있고 사실에 기반한 "
         "의견 기사 작성",
    backstory="당신은 {topic}에 대한 새로운 "
              "의견 기사를 작성하고 있습니다. "
              "콘텐츠 기획자가 제공한 개요와 "
              "주제 관련 맥락을 바탕으로 글을 씁니다. "
              "콘텐츠 기획자가 제시한 "
              "주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰을 제공하고 "
              "이를 콘텐츠 기획자가 제공한 "
              "정보로 뒷받침합니다. "
              "의견 기사에서 객관적 진술과 "
              "구별되는 개인적 의견을 명시합니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [16]:
editor = Agent(
    role="편집자",
    goal="주어진 블로그 글을 조직의 글쓰기 스타일에 "
         "맞게 편집",
    backstory="당신은 콘텐츠 작성자로부터 "
              "블로그 글을 받는 편집자입니다. "
              "블로그 글을 검토하여 저널리즘의 모범 사례를 "
              "따르고 있는지 확인하고, "
              "의견이나 주장을 제시할 때 "
              "균형 잡힌 관점을 제공하며, "
              "가능한 한 주요 논쟁적 주제나 "
              "의견을 피하도록 하는 것이 당신의 목표입니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

## Task(작업) 생성하기

- Task를 정의하고, `description`(설명), `expected_output`(예상 결과물), `agent`(수행 에이전트)를 제공합니다.


### Task: Plan

In [17]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드, 주요 인물, "
            "그리고 주목할 만한 뉴스를 우선순위화하기.\n"
        "2. 목표 독자층의 관심사와 pain point를 "
            "고려하여 파악하기.\n"
        "3. 서론, 핵심 요점, 행동 유도(CTA)를 포함한 "
            "상세한 콘텐츠 개요 개발하기.\n"
        "4. SEO 키워드와 관련 데이터 또는 출처 포함하기."
    ),
    expected_output="개요, 독자층 분석, "
        "SEO 키워드, 참고 자료를 포함한 "
        "포괄적인 콘텐츠 기획 문서."
        "결과물은 한국어로 생성될 예정입니다.",
    agent=planner,
)

### Task: Write

In [18]:
write = Task(
    description=(
        "1. 콘텐츠 기획을 활용하여 {topic}에 대한 "
            "매력적인 블로그 글 작성하기.\n"
        "2. SEO 키워드를 자연스럽게 포함하기.\n"
        "3. 섹션/부제목을 흥미롭게 적절한 "
            "이름으로 지정하기.\n"
        "4. 매력적인 서론, 통찰력 있는 본문, "
            "요약하는 결론으로 글을 구성하기.\n"
        "5. 문법 오류를 점검하고 "
            "브랜드의 톤앤매너와 일치하는지 확인하기.\n"
    ),
    expected_output="각 섹션이 2~3개의 문단으로 구성된, "
        "출판 준비가 완료된 마크다운 형식의 "
        "잘 작성된 블로그 글."
        "결과물은 한국어로 생성될 예정입니다.",
    agent=writer,
)

### Task: Edit

In [19]:
edit = Task(
    description=("주어진 블로그 글의 문법 오류를 검토하고 "
                 "브랜드의 톤앤매너와 일치하는지 확인하기."),
    expected_output="각 섹션이 2~3개의 문단으로 구성된, "
                    "출판 준비가 완료된 마크다운 형식의 "
                    "잘 작성된 블로그 글."
                    "결과물은 한국어로 생성될 예정입니다.",
    agent=editor
)

## Crew(팀) 생성하기

- Agent들로 구성된 팀을 생성합니다
- 해당 Agent들이 수행할 작업들을 전달합니다.
    - **참고**: *이 간단한 예시에서는* 작업들이 순차적으로 수행됩니다(즉, 서로 의존적임). 따라서 목록에서의 작업 _순서_가 _중요_합니다.
- `verbose=2`를 설정하면 실행의 모든 로그를 확인할 수 있습니다.


이 설정에서:
1. `planner`가 먼저 콘텐츠를 기획합니다
2. `writer`가 기획된 내용을 바탕으로 글을 작성합니다
3. `editor`가 최종적으로 작성된 글을 검토하고 편집합니다

In [20]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

In [21]:
result = crew.kickoff(inputs={"topic": "LLM(Large Language Model)을 이용한 지능형 에이전트 경쟁력 제고 방안"})

 [DEBUG]: == Working Agent: 콘텐츠 기획자
 [INFO]: == Starting Task: 1. LLM(Large Language Model)을 이용한 지능형 에이전트 경쟁력 제고 방안에 대한 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스를 우선순위화하기.
2. 목표 독자층의 관심사와 pain point를 고려하여 파악하기.
3. 서론, 핵심 요점, 행동 유도(CTA)를 포함한 상세한 콘텐츠 개요 개발하기.
4. SEO 키워드와 관련 데이터 또는 출처 포함하기.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer:

**지능형 에이전트 경쟁력 제고 방안: LLM(Large Language Model) 활용**

**1. 개요**

*서론:*  
지능형 에이전트는 현대 비즈니스에서 필수적인 도구로 자리잡고 있습니다. 특히, LLM(Large Language Model)의 발전은 이러한 에이전트의 능력을 획기적으로 높였습니다. 본 글에서는 LLM을 활용하여 지능형 에이전트의 경쟁력을 제고할 수 있는 여러 방안과 최신 트렌드를 살펴봅니다.

*핵심 요점:*  
- LLM의 최신 트렌드 및 발전: GPT-4, PaLM 2 등 최신 모델의 기능과 활용 사례.
- 주요 인물과 기업: OpenAI, 구글, 마이크로소프트 등의 기술적 기여.
- LLM 활용의 장점: 자연어 처리 능력 향상, 고객 서비스 자동화, 데이터 분석의 고도화.
- 실제 적용 사례: 산업별 LLM 적용 사례 소개.
- 경쟁력 제고 전략: 맞춤형 솔루션 개발, 지속적 학습 및 데이터 업데이트, 윤리적 AI 활용 방안.

*행동 유도(CTA):*  
독자들에게 LLM을 활용한 지능형 에이전트 솔루션 도입의 필요성을 강조하며, 관련 세미나 참석이나 기술 컨설팅 문의를 제안합니다.

**2. 독자층 분석**

- **목표 독자층:** IT 및 AI 기술 담당자, 비즈니스 전략가

In [23]:
from IPython.display import Markdown
Markdown(result)

```markdown
# LLM을 활용한 지능형 에이전트 경쟁력 강화 방안

## 서론: 미래를 여는 지능형 에이전트

지능형 에이전트는 현대 비즈니스에서 필수적인 도구로 자리잡고 있습니다. 특히, LLM(Large Language Model)의 발전은 이러한 에이전트의 능력을 획기적으로 높였습니다. 이러한 변화는 기업의 경쟁력을 높이고, 고객 경험을 향상시키는 중요한 요소로 작용하고 있습니다. 본 글에서는 LLM을 활용하여 지능형 에이전트의 경쟁력을 제고할 수 있는 여러 방안과 최신 트렌드를 살펴봅니다.

## LLM의 최신 트렌드와 발전

최근 LLM 기술은 GPT-4와 PaLM 2와 같은 첨단 모델의 등장으로 빠르게 발전하고 있습니다. 이들 모델은 자연어 처리 능력을 획기적으로 개선하여, 복잡한 언어 이해와 생성 작업을 수행할 수 있습니다. OpenAI, 구글, 마이크로소프트와 같은 주요 기업들은 이러한 기술 발전에 기여하며, 다양한 산업에 LLM을 적용하는 데 앞장서고 있습니다. 이러한 최신 트렌드는 기업들이 지능형 에이전트를 통해 혁신을 이끌어낼 수 있는 기회를 제공합니다.

## LLM 활용의 장점

LLM을 활용한 지능형 에이전트는 자연어 처리 능력을 크게 향상시킬 수 있습니다. 이를 통해 고객 서비스 자동화가 가능해지며, 고객의 요구에 빠르고 정확하게 응답할 수 있습니다. 또한, 데이터 분석의 고도화를 통해 더 깊이 있는 인사이트를 얻을 수 있어, 비즈니스 전략 수립에 큰 도움을 줄 수 있습니다. 이러한 장점은 기업이 경쟁에서 우위를 점할 수 있게 하는 중요한 요소가 됩니다.

## 실제 적용 사례: 다양한 산업에서의 성공적인 도입

LLM은 다양한 산업에서 그 활용 가능성을 입증하고 있습니다. 금융 분야에서는 고객 상담과 거래 분석에, 헬스케어에서는 환자 기록 분석 및 맞춤형 정보 제공에, 고객 서비스 분야에서는 24/7 자동 응대 시스템 구축에 활용되고 있습니다. 이러한 실제 사례들은 LLM이 비즈니스에 실질적인 가치를 제공할 수 있음을 보여줍니다.

## 경쟁력 제고 전략

LLM을 활용한 지능형 에이전트의 경쟁력을 높이기 위해서는 몇 가지 전략이 필요합니다. 먼저, 각 기업의 특성에 맞춘 맞춤형 솔루션을 개발해야 합니다. 또한, 지속적인 학습과 데이터 업데이트를 통해 최신 기술을 유지하는 것이 중요합니다. 마지막으로, 윤리적 AI 활용 방안을 마련하여 데이터 프라이버시와 보안을 강화해야 합니다.

## 결론: LLM으로 미래를 준비하라

LLM을 활용한 지능형 에이전트는 기업의 미래를 이끌어갈 핵심 도구입니다. 최신 AI 기술을 도입함으로써 기업은 경쟁력을 강화하고, 고객에게 더 나은 서비스를 제공할 수 있습니다. LLM을 활용한 솔루션 도입을 고려하는 기업들은 관련 세미나에 참석하거나 기술 컨설팅을 통해 구체적인 전략을 마련하는 것을 추천합니다. 이를 통해 AI 기술 혁신의 선두에 서고, 지속 가능한 성장을 이끌어 나갈 수 있을 것입니다.
```

This final answer is a well-structured and polished blog post in Markdown format, ready for publication. It follows the organization's writing style, maintains a balanced perspective, and avoids controversial statements.

- Display the results of your execution as markdown in the notebook.